Program that takes in an hdf5 file with 3d array of simulation and outputs the radial profile. Then attempts to fit it onto the analytic radial profile.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os
from scipy.optimize import minimize

Function takes in path for hdf5 file and outputs a 3d array with all the densities:

In [ ]:
# function takes in a path and outputs a 3d array of the density profile.

def file_to_density(path_name):
    hf = h5py.File(path_name, 'r')
    psi_real = np.array(hf['psiRe'])
    psi_im = np.array(hf['psiIm'])
    psi_real2 = psi_real**2
    psi_im2 = psi_im**2
    rho = (psi_real2 + psi_im2)
    return rho

Function rolls the density array so that the core is in the center of the image.

In [ ]:
#function takes in a cubic np density array and rolls it so tha the highest point is in the center.
#only to be used for the steady state images

def rho_roll(rho):
    rho_len = rho.shape[0]
    # finding the maximum value's index
    max_ind = np.where(rho == np.amax(rho))
    x_max = max_ind[0][0]
    y_max = max_ind[1][0]
    z_max = max_ind[2][0]
    signed_x_dist_to_cent = int(rho_len / 2) - x_max
    signed_y_dist_to_cent = int(rho_len / 2) - y_max
    signed_z_dist_to_cent = int(rho_len / 2) - z_max
    #moving the core to the center of the cube
    rho_n = np.roll(rho, signed_x_dist_to_cent, axis=0)
    rho_n2 = np.roll(rho_n, signed_y_dist_to_cent, axis=1)
    rho_n3 = np.roll(rho_n2, signed_z_dist_to_cent, axis=2)
    rho = rho_n3
    return rho

A helper function that computes the distance between two 3d points.

In [ ]:
# function that calculates distances between two 3d points:

def get_distance(a, b):
    return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2 + (a[2]- b[2])**2)

Function returns a dictionary with the radii as keys and a list of the densities as values.

In [ ]:
#function that takes in a 3d array with highest point at the center and calculates
#the density as a function of radius fromt the center of the core. returns as a dictionary with
#distances as keys and a list of the densities as the value.

def radial_density(rho):
    rho_len = rho.shape[0]
    center = int(rho_len / 2)
    radii_den = {}
    for i in range(0, rho_len):
        for j in range(0, rho_len):
            for k in range(0, rho_len):
                dist = get_distance([center, center, center], [i, j, k])
                dens = rho[i][j][k]
                if dist not in radii_den:
                    radii_den[dist] = [dens]
                else:
                    radii_den[dist].append(dens)
    return radii_den

# testing
a = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
b = np.array([[[1, 4], [3, 4]], [[5, 6], [7, 8]]])
dict = radial_density(a)
print(len(dict))
dist = np.fromiter(dict.keys(), dtype=float)
print(dist)


This function takes in a dictionary of radii and densities and returns two numpy arrays, one with radii and the other with average densities.

In [ ]:
#function that plots radial density. Takes in a dictionary with densities as keys
# and the densities as values and outputs two np arrays, one with the distances 
# and one with the ---average--- density at that radial distance.

def rad_avg_den(dict, scale_factor):
    dict_len = len(dict)
    dist = np.fromiter(dict.keys(), dtype=float)
    avgs = np.zeros(dict_len)
    for x in range(0, dict_len):
        dens = np.array(dict[dist[x]])
        avg = np.mean(dens)
        avgs[x] = avg
    return dist * scale_factor, avgs
# testing
#res = rad_avg_den(dict)
#print(res[0])
#print(res[1])

Function returns two numpy arrays, one with the radial positions, and the other with all the densities, even the ones with the doubled up radial positions.

In [ ]:
# function that takes in dictionary and prints two np arrays, one with the points
# and another with the corresponding densities, including doubles.

def rad_den(dict):
    dict_len = len(dict)
    dist_list = np.fromiter(dict.keys(), dtype=float)
    dist = []
    dens = []
    for x in range(0, dict_len):
        curr_dist = dist_list[x]
        list_len = len(dict[dist_list[x]])
        for i in range(0, list_len):
            dist.append(curr_dist)
            dens.append(dict[dist_list[x]][i])
    dist = np.asarray(dist)
    dens = np.asarray(dens)
    return dist * (20/128), dens

In [ ]:
# from path to distance and average density arrays
def file_to_rad_avg_den(path):
    rho = file_to_density(path)
    rho = rho_roll(rho)
    scale_factor = 20 / rho.shape[0]
    rad_den = radial_density(rho)
    return rad_avg_den(rad_den, scale_factor)

In [ ]:
# from path to distance and unaveraged density arrays
def file_to_rad_den(path):
    rho = file_to_density(path)
    rho = rho_roll(rho)
    radial_den = radial_density(rho)
    return rad_den(radial_den)

Testing all the functions to see if they work:

In [ ]:
#testing the file to density function
path = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/output/fInfL20T4n400r128/snap0300.h5'
res = file_to_rad_avg_den(path)
res2 = file_to_rad_den(path)

In [ ]:
print(len(res[0]))
print(10e4)

In [ ]:
fig_path = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/output/fInfL20T4n400r128'
fig_name = 'pic300'
plt.scatter(res[0], res[1])
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Radial Distance (kpc)')
plt.ylabel('Density (Msun / (kpc)^3)')
plt.title('Radial Distance vs. Density')
plt.ylim(10**4, 10**(10.5))
plt.xlim(10**(-0.5), 10**(2.2))
plt.savefig(fig_path + "/" + fig_name)
plt.clf()

In [ ]:
#file to picture
def file_to_pic(path_to_dir, fig_num, f_str):
    res = file_to_rad_avg_den(path_to_dir + '/snap' + fig_num  + '.h5')
    plt.scatter(res[0], res[1])
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Radial Distance (kpc)')
    plt.ylabel('Density (Msun / (kpc)^3)')
    plt.title('Radial Distance vs. Density: f = ' + f_str)
    plt.ylim(10**4, 10**(10.5))
    plt.xlim(10**(-1), 10**(2.2))
    plt.savefig(path_to_dir + "/" + f_str + 'pic' + fig_num)
    plt.clf()

In [ ]:
#testing
file_to_pic('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/output/fInfL20T4n400r128', '0350', 'Inf')
file_to_pic('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/output/f1L20T4n400r128', '0350', '1')
file_to_pic('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/output/f2L20T4n400r128', '0350', '2')
file_to_pic('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/output/f4L20T4n400r128', '0350', '4')


Function fitting

In [ ]:
test_path = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/output/f1L20T4n400r128/snap0350.h5'
sample_data = file_to_rad_avg_den(test_path)
print(sample_data[0])

In [ ]:
#cuts off a distance array such that it only has elements < a certain length

def dist_cutoff(dist, dist_lim):
    distance = []
    for x in dist:
        if (x < dist_lim):
            distance.append(x)
    return np.asarray(distance)

In [ ]:
#cuts off a density array at a given number of elements
#not in place

def dens_cutoff(dens, n_elements):
    densities = np.zeros(n_elements)
    for i in range(0, n_elements):
        densities[i] = dens[i]
    return densities

In [ ]:
#analytic profile of the core

def an_prof(r, rc):
    rho = ((1.9*10**7) / (rc**4)) #*((1/0.1)**2)
    term = (1 + 0.091*(r/rc)**2)**-8
    return rho * term

In [ ]:
#X2 function

def X2_fun(rc, dist, dens, dist_lim):
    r = dist_cutoff(dist, dist_lim)
    densities = dens_cutoff(dens, len(r))
    X2 = (((an_prof(r, rc)) - densities)**2)
    indices = [0, len(X2)-1]
    return X2[indices].sum()

In [ ]:
#radial-density fitting function
#params are two nparrays
#analytic radial profile function

def fit_profile(dist, dens, dist_lim):
    dist1 = np.sort(dist)
    dens1 = (-1)*(np.sort(dens*(-1)))
    params = (dist1, dens1, dist_lim)
    x0 = np.asarray(3)
    result = minimize(X2_fun, x0, args = params)
    return result

test_result = fit_profile(sample_data[0], sample_data[1], 10**(-1.1))
print(test_result)

#testing
rc = test_result.x[0]
dist_lim = 0.65
r = sample_data[0]
densities = sample_data[1]
plt.scatter(r, densities)
plt.plot(r, an_prof(r, rc))
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Radial Distance (kpc)')
plt.ylabel('Density (Msun / (kpc)^3)')
plt.title('Radial Distance vs. Density')
plt.ylim(10**4, 10**(12.5))
plt.xlim(10**(-2), 10**(1.4))
textstr = 'rc = ' + "%.2f" % rc
plt.legend(['Analytic Profile, rc = ' + "%.2f" % rc,'Simulation'])
plt.show()

In [ ]:
def file_to_pic_curve_fit(path, file, dist_lim):
    arrs = file_to_rad_avg_den(path + '/' + file)
    dist = arrs[0]
    dens = arrs[1]
    dist1 = np.sort(dist)
    dens1 = (-1)*(np.sort(dens*(-1)))
    res = fit_profile(dist1, dens1, dist_lim)
    rc = res.x[0]
    plt.scatter(dist1, dens1)
    plt.plot(dist1, an_prof(dist1, rc))
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Radial Distance (kpc)')
    plt.ylabel('Density (Msun / (kpc)^3)')
    plt.title('Radial Distance vs. Density')
    plt.ylim(10**4, 10**(12.5))
    plt.xlim(10**(-2), 10**(1.4))
    textstr = 'rc = ' + "%.2f" % rc
    plt.legend(['Analytic Profile, rc = ' + "%.2f" % rc,'Simulation'])
    plt.savefig(path + '/pic' + file + '.png')
    

In [ ]:
file_to_pic_curve_fit('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/output/f1L20T4n400r128', 'snap0350.h5', .1)

In [ ]:
file_to_pic_curve_fit('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/output/fInfL20T4n400r128', 'snap0350.h5', .65)

In [ ]:
file_to_pic_curve_fit('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/output/f2L20T4n400r128', 'snap0350.h5', 0.65)

In [ ]:
file_to_pic_curve_fit('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/output/f4L20T4n400r128', 'snap0350.h5', 0.65)

In [ ]:
file_to_pic_curve_fit('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/output/f1.5L20T4n400r128', 'snap0350.h5', 0.65)

In [ ]:
# trying to write the radial profile to hdf5 file along with minimized rc parameter
def path_to_radial_h5(path_to_folder, snap_number):
    result = file_to_rad_avg_den(path_to_folder + '/snap' + str(snap_number).zfill(4) + '.h5')
    dist = np.sort(result[0])
    dens = (-1)*(np.sort(result[1]*(-1)))
    dist_lim = 0.65
    params = (dist, dens, dist_lim)
    x0 = np.asarray(2)
    res = minimize(X2_fun, x0, args = params)
    rc = res.x[0]
    
    hf = h5py.File(path_to_folder + '/' + 'radial' + str(snap_number) + '.h5', 'w')
    hf.create_dataset('distances', data=dist)
    hf.create_dataset('densities', data=dens)
    hf.create_dataset('rc', data=rc)
    hf.close()

In [ ]:
# writes radial h5 files for all files in folder:
def radial_h5_folder(path, n_snap):
    dir = path + '/rad_files'
    if not os.path.exists(dir):
        os.mkdir(dir)
    for i in range(0, n_snap + 1):
        result = file_to_rad_avg_den(path + '/snap' + str(i).zfill(4) + '.h5')
        dist = np.sort(result[0])
        dens = (-1)*(np.sort(result[1]*(-1)))
        dist_lim = 0.65
        params = (dist, dens, dist_lim)
        x0 = np.asarray(2)
        res = minimize(X2_fun, x0, args = params)
        rc = res.x[0]
        # saving as file
        hf = h5py.File(dir + '/' + 'radial' + str(i).zfill(4) + '.h5', 'w')
        hf.create_dataset('distances', data=dist)
        hf.create_dataset('densities', data=dens)
        hf.create_dataset('rc', data=rc)
        hf.close()
        
#testing
folder = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/fInfL20T4n40r64'
radial_h5_folder(folder, 40)

In [ ]:
path_to_radial_h5('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/output/fInfL20T4n40r64', 40)

In [3]:
#analyzing data for f = 1.125
path = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f1.125L20T4n40r256'

for i in range(0, 41):    
    hf = h5py.File(path + '/radial' + str(i).zfill(4) + '.h5', 'r')
    dist = np.array(hf['distances'])
    dens = np.array(hf['densities'])
    plt.scatter(dist, dens)
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Radial Distance (kpc)')
    plt.ylabel('Density (Msun / (kpc)^3)')
    plt.title('Radial Distance vs. Density')
    plt.ylim(10**4, 10**(12.5))
    plt.xlim(10**(-2), 10**(1.4))
    plt.savefig(path + '/pics/pic' + str(i).zfill(4) + '.png')
    plt.clf()


<Figure size 432x288 with 0 Axes>

Trying to plot enclosed mass as a function of radius in the case of the analytic profile (assuming a spherical object)

In [ ]:
def enclosed_mass(r, dens):
    V = (4/3)*np.pi*r**3
    return V*dens

In [ ]:
r = np.linspace(0, 15, 1000000)

dens = an_prof(r, 0.18)
m = enclosed_mass(r,dens)
m = m*15/1000000
sum = 0
menc = np.zeros(1000000)
for i in range(1, 1000000):
    menc[i] = sum
    sum = sum + m[i]
plt.plot(r, menc)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Radial Distance (kpc)')
plt.ylabel('M(enclosed) (Msun)')
plt.title('Enclosed Mass vs. Distance')
plt.ylim(10**(5), 10**(11))
plt.xlim(10**(-3), 10**(1))



In [ ]:
#plotting radial profiles of different densities

#naming file paths
path64 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f2L20T4n400r64/radial0400.h5'
path128 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f2L20T4n400r128/radial0400.h5'
path256 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f2L20T4n400r256/radial0400.h5'
path400 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f2L20T4n400r400/radial0400.h5'

#opening files and assigning to arrays
hf64 = h5py.File(path64, 'r')
hf128 = h5py.File(path128, 'r')
hf256 = h5py.File(path256, 'r')
hf400 = h5py.File(path400, 'r')

dist64 = np.array(hf64['distances'])
dens64 = np.array(hf64['densities'])
dist128 = np.array(hf128['distances'])
dens128 = np.array(hf128['densities'])
dist256 = np.array(hf256['distances'])
dens256 = np.array(hf256['densities'])
dist400 = np.array(hf400['distances'])
dens400 = np.array(hf400['densities'])

plt.plot(dist64, dens64)
plt.plot(dist128, dens128)
plt.plot(dist256, dens256)
plt.plot(dist400, dens400)

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Radial Distance (kpc)')
plt.ylabel('Density (Msun / (kpc)^3)')
plt.title('Radial Distance vs. Density')
plt.ylim(10**4, 10**(12.5))
plt.xlim(10**(-2), 10**(1.4))

plt.xlabel('Radial Distance (kpc)')
plt.ylabel('Density (Msun/ kpc^3)')
plt.title('Radius vs. Density')
plt.legend(['r = 64', 'r = 128', 'r = 256', 'r = 400'])

plt.savefig('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/smooth_convergence_testf2.png')

In [ ]:
#plotting radial profiles of different densities

#naming file paths
path64 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f2L20T4n400r64/radial0400.h5'
path128 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f2L20T4n400r128/radial0400.h5'
path256 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f2L20T4n400r256/radial0400.h5'
path400 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f2L20T4n400r400/radial0400.h5'

#opening files and assigning to arrays
hf64 = h5py.File(path64, 'r')
hf128 = h5py.File(path128, 'r')
hf256 = h5py.File(path256, 'r')
hf400 = h5py.File(path400, 'r')

dist64 = np.array(hf64['distances'])
dens64 = np.array(hf64['densities'])
dist128 = np.array(hf128['distances'])
dens128 = np.array(hf128['densities'])
dist256 = np.array(hf256['distances'])
dens256 = np.array(hf256['densities'])
dist400 = np.array(hf400['distances'])
dens400 = np.array(hf400['densities'])

plt.scatter(dist64, dens64, s = 0.25)
plt.scatter(dist128, dens128, s = 0.25)
plt.scatter(dist256, dens256, s = 0.25)
plt.scatter(dist400, dens400, s = 0.25)

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Radial Distance (kpc)')
plt.ylabel('Density (Msun / (kpc)^3)')
plt.title('Radial Distance vs. Density')
plt.ylim(10**4, 10**(12.5))
plt.xlim(10**(-2), 10**(1.4))

plt.xlabel('Radial Distance (kpc)')
plt.ylabel('Density (Msun/ kpc^3)')
plt.title('Radius vs. Density')
plt.legend(['r = 64', 'r = 128', 'r = 256', 'r = 400'])


plt.savefig('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/scatter_convergence_testf2.png')

In [ ]:
#plotting radial profiles of different densities

#naming file paths
path1 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f1L20T4n400r256/radial0400.h5'
path1125 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f1.125L20T4n40r256/radial0040.h5'
path11875 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f1.1875L20T4n40r256/radial0040.h5'
path125 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f1.25L20T4n40r256/radial0040.h5'
path15 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f1.5L20T4n400r256/radial0400.h5'
path175 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f1.75L20T4n40r256/radial0040.h5'
path2 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f2L20T4n400r256/radial0400.h5'
path4 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f4L20T4n400r256/radial0400.h5'
pathInf ='/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/fInfL20T4n400r256/rad_filesradial0400.h5'

#opening files and assigning to arrays
hf1 = h5py.File(path1, 'r')
hf1125 = h5py.File(path1125, 'r')
hf11875 = h5py.File(path11875, 'r')
hf125 = h5py.File(path125, 'r')
hf15 = h5py.File(path15, 'r')
hf175 = h5py.File(path175, 'r')
hf2 = h5py.File(path2, 'r')
hf4 = h5py.File(path4, 'r')
hfInf = h5py.File(pathInf, 'r')

dist1 = np.array(hf1['distances'])
dens1 = np.array(hf1['densities'])

dist1125 = np.array(hf1125['distances'])
dens1125 = np.array(hf1125['densities'])

dist11875 = np.array(hf11875['distances'])
dens11875 = np.array(hf11875['densities'])

dist125 = np.array(hf125['distances'])
dens125 = np.array(hf125['densities'])

dist15 = np.array(hf15['distances'])
dens15 = np.array(hf15['densities'])

dist175 = np.array(hf175['distances'])
dens175 = np.array(hf175['densities'])

dist2 = np.array(hf2['distances'])
dens2 = np.array(hf2['densities'])

dist4 = np.array(hf4['distances'])
dens4 = np.array(hf4['densities'])

distInf = np.array(hfInf['distances'])
densInf = np.array(hfInf['densities'])

plt.scatter(dist1, dens1, s = 0.25)
plt.scatter(dist1125, dens1125, s = 0.25)
plt.scatter(dist11875, dens11875, s = 0.25)
plt.scatter(dist125, dens125, s = 0.25)
plt.scatter(dist15, dens15, s = 0.25)
plt.scatter(dist175, dens175, s = 0.25)
plt.scatter(dist2, dens2, s = 0.25)
plt.scatter(dist4, dens4, s = 0.25)
plt.scatter(distInf, densInf, s = 0.25)

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Radial Distance (kpc)')
plt.ylabel('Density (Msun / (kpc)^3)')
plt.title('Radial Distance vs. Density')
plt.ylim(10**4, 10**(12.5))
plt.xlim(10**(-2), 10**(1.4))

plt.xlabel('Radial Distance (kpc)')
plt.ylabel('Density (Msun/ kpc^3)')
plt.title('Radius vs. Density')
plt.legend(['f = 1', 'f = 1.125', 'f = 1.1875', 'f = 1.25', 'f = 1.5', 'f = 1.75', 'f = 2', 'f = 4', 'f = Inf'])

plt.savefig('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f_r256_full.png', dpi = 200)

In [ ]:
#plotting radial profiles of different densities

#naming file paths
path1 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f1L20T4n400r400/radial0200.h5'
#path125 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f1.25L20T4n40r400/radial0040.h5'
path15 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f1.5L20T4n400r400/radial0200.h5'
#path175 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f1.75L20T4n40r400/radial0040.h5'
path2 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f2L20T4n400r400/radial0400.h5'
path4 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f4L20T4n400r400/radial0400.h5'
pathInf ='/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/fInfL20T4n400r400/radial0400.h5'

#opening files and assigning to arrays
hf1 = h5py.File(path1, 'r')
#hf125 = h5py.File(path125, 'r')
hf15 = h5py.File(path15, 'r')
#hf175 = h5py.File(path175, 'r')
hf2 = h5py.File(path2, 'r')
hf4 = h5py.File(path4, 'r')
hfInf = h5py.File(pathInf, 'r')

dist1 = np.array(hf1['distances'])
dens1 = np.array(hf1['densities'])

#dist125 = np.array(hf125['distances'])
#dens125 = np.array(hf125['densities'])

dist15 = np.array(hf15['distances'])
dens15 = np.array(hf15['densities'])

#dist175 = np.array(hf175['distances'])
#dens175 = np.array(hf175['densities'])

dist2 = np.array(hf2['distances'])
dens2 = np.array(hf2['densities'])

dist4 = np.array(hf4['distances'])
dens4 = np.array(hf4['densities'])

distInf = np.array(hfInf['distances'])
densInf = np.array(hfInf['densities'])

plt.scatter(dist1, dens1, s = 0.25)
#plt.scatter(dist125, dens125, s = 0.25)
plt.scatter(dist15, dens15, s = 0.25)
#plt.scatter(dist175, dens175, s = 0.25)
plt.scatter(dist2, dens2, s = 0.25)
plt.scatter(dist4, dens4, s = 0.25)
plt.scatter(distInf, densInf, s = 0.25)

plt.xscale('log')
plt.yscale('log')
plt.xlabel('Radial Distance (kpc)')
plt.ylabel('Density (Msun / (kpc)^3)')
plt.title('Radial Distance vs. Density')
plt.ylim(10**4, 10**(12.5))
plt.xlim(10**(-2), 10**(1.4))

plt.xlabel('Radial Distance (kpc)')
plt.ylabel('Density (Msun/ kpc^3)')
plt.title('Radius vs. Density')
plt.legend(['f = 1', 'f = 1.5', 'f = 2', 'f = 4', 'f = Inf'])

plt.savefig('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f_r400.png', dpi = 200)

In [ ]:
#plotting radial profiles of different densities

#naming file paths
path64 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f2L20T4n400r64/radial0400.h5'
path128 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f2L20T4n400r128/radial0400.h5'
path256 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f2L20T4n400r256/radial0400.h5'
path400 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f2L20T4n400r400/radial0400.h5'

#opening files and assigning to arrays
hf64 = h5py.File(path64, 'r')
hf128 = h5py.File(path128, 'r')
hf256 = h5py.File(path256, 'r')
hf400 = h5py.File(path400, 'r')

dist64 = np.array(hf64['distances'])
dens64 = np.array(hf64['densities'])
dist128 = np.array(hf128['distances'])
dens128 = np.array(hf128['densities'])
dist256 = np.array(hf256['distances'])
dens256 = np.array(hf256['densities'])
dist400 = np.array(hf400['distances'])
dens400 = np.array(hf400['densities'])

#plt.scatter(dist64, dens64)
#plt.scatter(dist128, dens128)
#plt.scatter(dist256, dens256)
#plt.scatter(dist400, dens400)


#plt.xlabel('Radial Distance (kpc)')
#plt.ylabel('Density (Msun/ kpc^3)')
#plt.title('Radius vs. Density')
#plt.legend(['r = 64', 'r = 128', 'r = 256', 'r = 400'])


fig, axs = plt.subplots(2, 2)
axs[0, 0].scatter(dist64, dens64)
axs[0, 0].set_title('R = 64')
axs[0, 0].loglog()
axs[0, 0].set_ylim(10**4, 10**(12.5))
axs[0, 0].set_xlim(10**(-2), 10**(1.4))
#axs[0, 0].xlabel('Radial Distance (kpc)')
#axs[0, 0].ylabel('Density (Msun / (kpc)^3)')
#axs[0, 0].title('Radial Distance vs. Density')


axs[0, 1].scatter(dist128, dens128)
axs[0, 1].set_title('R = 128')
axs[0, 1].loglog()
axs[0, 1].set_ylim(10**4, 10**(12.5))
axs[0, 1].set_xlim(10**(-2), 10**(1.4))

axs[1, 0].scatter(dist256, dens256)
axs[1, 0].set_title('R = 256')
axs[1, 0].loglog()
axs[1, 0].set_ylim(10**4, 10**(12.5))
axs[1, 0].set_xlim(10**(-2), 10**(1.4))

axs[1, 1].scatter(dist400, dens400)
axs[1, 1].set_title('R = 400')
axs[1, 1].loglog()
axs[1, 1].set_ylim(10**4, 10**(12.5))
axs[1, 1].set_xlim(10**(-2), 10**(1.4))

#fig.suptitle('Density Profiles', fontsize=16, pad = 20)
for ax in axs.flat:
    ax.set(xlabel='Radius (kpc)', ylabel='Density (Msun/kpc^3)')
plt.tight_layout()
plt.savefig('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/convergence_test.png')

In [ ]:
#plotting radial profiles of different densities

#naming file paths

path1 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f1L20T4n400r256/radial0400.h5'
path15 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f1.5L20T4n400r256/radial0400.h5'
path2 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f2L20T4n400r256/radial0400.h5'
path4 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f4L20T4n400r256/radial0400.h5'

#opening files and assigning to arrays
hf15 = h5py.File(path15, 'r')
hf1 = h5py.File(path1, 'r')
hf2 = h5py.File(path2, 'r')
hf4 = h5py.File(path4, 'r')

dist15 = np.array(hf15['distances'])
dens15 = np.array(hf15['densities'])
dist1 = np.array(hf1['distances'])
dens1 = np.array(hf1['densities'])
dist2 = np.array(hf2['distances'])
dens2 = np.array(hf2['densities'])
dist4 = np.array(hf4['distances'])
dens4 = np.array(hf4['densities'])

#plt.scatter(dist64, dens64)
#plt.scatter(dist128, dens128)
#plt.scatter(dist256, dens256)
#plt.scatter(dist400, dens400)


#plt.xlabel('Radial Distance (kpc)')
#plt.ylabel('Density (Msun/ kpc^3)')
#plt.title('Radius vs. Density')
#plt.legend(['r = 64', 'r = 128', 'r = 256', 'r = 400'])


fig, axs = plt.subplots(2, 2)
axs[0, 1].scatter(dist15, dens15)
axs[0, 1].set_title('f = 1.5')
axs[0, 1].loglog()
axs[0, 1].set_ylim(10**4, 10**(12.5))
axs[0, 1].set_xlim(10**(-2), 10**(1.4))
#axs[0, 0].xlabel('Radial Distance (kpc)')
#axs[0, 0].ylabel('Density (Msun / (kpc)^3)')
#axs[0, 0].title('Radial Distance vs. Density')


axs[0, 0].scatter(dist1, dens1)
axs[0, 0].set_title('f = 1')
axs[0, 0].loglog()
axs[0, 0].set_ylim(10**4, 10**(12.5))
axs[0, 0].set_xlim(10**(-2), 10**(1.4))

axs[1, 0].scatter(dist2, dens2)
axs[1, 0].set_title('f = 2')
axs[1, 0].loglog()
axs[1, 0].set_ylim(10**4, 10**(12.5))
axs[1, 0].set_xlim(10**(-2), 10**(1.4))

axs[1, 1].scatter(dist4, dens4)
axs[1, 1].set_title('f = 4')
axs[1, 1].loglog()
axs[1, 1].set_ylim(10**4, 10**(12.5))
axs[1, 1].set_xlim(10**(-2), 10**(1.4))

#fig.suptitle('Density Profiles', fontsize=16, pad = 20)
for ax in axs.flat:
    ax.set(xlabel='Radius (kpc)', ylabel='Density (Msun/kpc^3)')
plt.tight_layout()
plt.savefig('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/radial_comparison_r256.png')

In [ ]:
#takes in a folder and returns an array of its max density as a function of time.
def max_rad(folder_path, snap_num):
    max_den = np.zeros(snap_num + 1)
    for i in range(0, snap_num + 1):
        hf = h5py.File(folder_path + '/radial' + str(i).zfill(4) + '.h5', 'r')
        den = np.array(hf['densities'])
        max_den[i] = den[0]
    return max_den



In [ ]:
#testing function
pathf11875 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f1.1875L20T4n40r256'
max_den = max_rad(pathf11875, 40)
time = np.linspace(0,40, 41)
plt.scatter(time, max_den)
plt.yscale('log')
plt.xlabel('Time')
plt.ylabel('Max Density (Msun / (kpc)^3)')
plt.title('Time vs. Max Density')

plt.savefig('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/pics/max_den/f11875r256.png', dpi = 200)

In [ ]:
#testing function
pathInf = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/fInfL20T4n400r256'
max_den = max_rad(pathInf, 40)
time = np.linspace(0,40, 41)
plt.scatter(time, max_den)
plt.yscale('log')
plt.xlabel('Time')
plt.ylabel('Max Density (Msun / (kpc)^3)')
plt.title('Time vs. Max Density')

plt.savefig('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/pics/max_den/fInfr256.png', dpi = 200)

In [ ]:
#testing function
path1 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f1L20T4n400r256'
max_den = max_rad(path1, 40)
time = np.linspace(0,40, 41)
plt.scatter(time, max_den)
plt.yscale('log')
plt.xlabel('Time')
plt.ylabel('Max Density (Msun / (kpc)^3)')
plt.title('Time vs. Max Density')

plt.savefig('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/pics/max_den/f1r256.png', dpi = 200)

In [ ]:
path2 = '/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/rad_files/f1.125L20T4n40r256'
max_den = max_rad(path2, 40)
time = np.linspace(0,40, 41)
plt.scatter(time, max_den)
plt.yscale('log')
plt.xlabel('Time')
plt.ylabel('Max Density (Msun / (kpc)^3)')
plt.title('Time vs. Max Density')

plt.savefig('/mnt/c/Users/Noah Notis/Dropbox/USRP2020_Noah/src/FDM2020/pics/max_den/f1125r256.png', dpi = 200)